In [1]:
using DifferentialEquations
using Plots

In [42]:
#Manually create repressilator to test
function oscillator(du, u, p, t)
    I = [0.1, 0, 0]
    k = 1

    du[1] = k*(phi(-u[3]) + I[1] - u[1])
    du[2] = k*(phi(-u[1]) + I[2] - u[2])
    du[3] = k*(phi(-u[2]) + I[3] - u[3])
end

tspan = [0, 100] #Span of time to integrate for
savetimes = LinRange(0, 100, 100) #linear space of points until final time
u0 = fill(0.1, N) #cannot change, can be learned
p = [] #empty for now, can change to include parameters to be varied later
prob = ODEProblem(oscillator, u0, tspan, p)
sol = solve(prob, Rosenbrock23(), reltol=1e-3, abstol=1e-6, saveat=savetimes)

┌ Warning: dt(1.4210854715202004e-14) <= dtmin(1.4210854715202004e-14) at t=0.004832143893210033, and step error estimate = 78.16594331724944. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase C:\Users\Charlotte\.julia\packages\SciMLBase\kTUaf\src\integrator_interface.jl:599


retcode: DtLessThanMin
Interpolation: 1st order linear
t: 1-element Vector{Float64}:
 0.0
u: 1-element Vector{Vector{Float64}}:
 [0.1, 0.1, 0.1]

In [43]:
#Manually specify an oscillator (repressilator from Alon et al)
N = 3
W = [[1, 0, -1], [-1, 1, 0], [0, -1, 1]] #Notation is INCOMING
I = [0, 0, 0]

phi(x) = 1/(exp(x) - 1) #Helper function
#Given N, W, I create a ODE function and run it
function hiscock_model(N, W, I, endtime=100, resolution=200)
    function odes(du, u, p, t)
        k = 1 #set to 1 for now, can be learned
        for i in 1:N
            du[i] = k*(phi(transpose(W[i])*u) + I[i] - u[i])
        end
    end

    tspan = [0, endtime] #Span of time to integrate for
    savetimes = LinRange(0, endtime, resolution) #linear space of points until final time
    u0 = fill(0.1, N) #cannot change, can be learned
    p = [] #empty for now, can change to include parameters to be varied later
    prob = ODEProblem(odes, u0, tspan, p)
    sol = solve(prob, Rosenbrock23(), reltol=1e-3, abstol=1e-6, saveat=savetimes)
    println(length(sol.u))
    return sol
end

sol = hiscock_model(N, W, I, 1000, 10)

1


┌ Warning: Instability detected. Aborting
└ @ SciMLBase C:\Users\Charlotte\.julia\packages\SciMLBase\kTUaf\src\integrator_interface.jl:606


retcode: Unstable
Interpolation: 1st order linear
t: 1-element Vector{Float64}:
 0.0
u: 1-element Vector{Vector{Float64}}:
 [0.1, 0.1, 0.1]

In [24]:
function plot_solution(sol)
end

1-element Vector{Vector{Float64}}:
 [0.1, 0.1, 0.1]